In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os   
import pandas as pd
import datetime 
import math
import time
from sklearn.linear_model import LinearRegression

# Threshold info
thresholds_dict = {
    'annual_threshold_tasmax_high' : {'variable': 'tasmax', 'value': 35, 'min_max_val': 'greater_than', 'b_consecutive': False},
    'annual_threshold_tasmin_high' : {'variable': 'tasmin', 'value': 35, 'min_max_val': 'greater_than', 'b_consecutive': False},
    'annual_threshold_tasmax_low' : {'variable': 'tasmax', 'value': 0, 'min_max_val': 'less_than', 'b_consecutive': False},
    'annual_threshold_tasmin_low' : {'variable': 'tasmin', 'value': 0, 'min_max_val': 'less_than', 'b_consecutive': False},
    # For now keep heat index to maximum values and number of residence days there
    'annual_threshold_heatindex_high': {'variable': 'heatindex', 'value': 100, 'min_max_val': 'greater_than', 'b_consecutive': False},
    'annual_threshold_precipitation_high': {'variable': 'pr', 'value': 1, 'min_max_val': 'greater_than', 'b_consecutive': False},
    'annual_threshold_precipitation_low': {'variable': 'pr', 'value': 1e-3, 'min_max_val': 'less_than', 'b_consecutive': False},
    'annual_threshold_consecutive_precipitation_high': {'variable': 'pr', 'value': 1, 'min_max_val': 'greater_than', 'b_consecutive': True},
    'annual_threshold_consecutive_precipitation_low': {'variable': 'pr', 'value': 1e-3, 'min_max_val': 'less_than', 'b_consecutive': True},
}

# Loading Local Information
location_name = 'Golden, CO'
coord = (39.74256572165156, -105.16857014167543)
location_name = 'Langley Research Center'
coord = (37.088061583314044, -76.3797854741248)
'''average_cold = -1.33
average_hot = 15.37
average_cold = 0.0
average_hot = 0.0'''
bool_all_files = True
path = 'NEX-GDDP-CMIP6/'

# Choose between all files or just a select few
if bool_all_files:
    # In the future this will be a os.listdir(), for now manually enter names
    test_list = os.listdir('NEX-GDDP-CMIP6')
    model_experiment_names = []
    for item in test_list:
        if 'download' in item:
            continue
        elif '.' in item:
            continue
        else:
            model_experiment_names.append(item)

else:
    model_experiment_names = ['EC-Earth3']

experiments = ['ssp245', 'ssp585']
model_experiment_names.sort()
print(model_experiment_names)


['ACCESS-CM2']


In [2]:
from load_model import load_model
from analyze_model import analyze_model
from graphing import *

# Getting the list of model names
model_list = []
for model_name in model_experiment_names:
    for experiment in experiments:
        model_to_read = load_model(path+model_name, model_name+'-'+experiment, experiment)
        model_list.append(model_to_read)

print('Starting Analysis Loop')
for item in model_list:
    # Compiling the data
    item.compile_data(coord)

    # Analyzing the Data
    try:
        analyze_model(item, thresholds_dict)
        path = 'output_data/'
        item.df_decade.to_excel(path+item.model_name+'_decade_values.xlsx')
    except: 
        print('Cannot analyze model: ', item.model_name, ' insufficient data')

    # Graphing the data
    variable_list = item.df_decade.columns
    for variable in variable_list:
        if 'Days' in variable:
            unit = 'Days per Year'
        elif 'Frequency' in variable:
            unit = 'Average Occurrences per Year'
        elif 'Duration' in variable:
            unit = 'Days'
        elif ('temperature' in variable) or ('Temperature' in variable):
            unit = 'Temperature (°C)'
        elif ('precipitation' in variable) or ('Precipitation' in variable):
            unit = 'Precipitation (inches)'
        else:
            unit = 'Not Assigned'
        graph_write_decade_variable(item, variable, unit)    
    print('Graphing Complete')

Starting Analysis Loop
Data Compiling for  ACCESS-CM2-ssp245
Data Compiled!
Analysis Complete
Graphing Complete
Data Compiling for  ACCESS-CM2-ssp585
Data Compiled!
Analysis Complete
Graphing Complete
